In [ ]:
##############################################################################
#on the login node after loging into gpuvolta node
module purge
module load cuda/10.1
module load julia/1.6.1
export LD_PRELOAD=/usr/lib64/libpthread.so.0
JULIA_DEBUG=CUDA JULIA_CUDA_USE_BINARYBUILDER=false julia --project
#############################################################################
using Printf
using Oceananigans
using Oceananigans.Utils
using Oceananigans.Units: minute, minutes, hour
using Oceananigans.Grids: nodes
using Oceananigans.Diagnostics
using Oceananigans.OutputWriters: JLD2OutputWriter, FieldSlicer, TimeInterval
using Oceananigans.Diagnostics: accurate_cell_advection_timescale


#number of grid spacing in south,north and vertical direction
const Nx=256
const Ny=256
const Nz=64

#Length of grid in south, north and vertical direction
const Lx=4
const Ly=4
const Lz=0.1

#Vetrical temperature gradient
const dTz = 70

#gravitational acceleration
const g=300

const R0=1
const T0=30
const Factor_T =1e-6
const Factor_V=1e-8

#Constructing the grid

const S = 2.0 # Stretching factor
hyperbolically_spaced_nodes(k) = -Lz-Lz*(tanh(S * ( (-(k-Nz) ) / Nz - 1)) / tanh(S))
computational_grid = VerticallyStretchedRectilinearGrid(size = (Nx, Ny, Nz), 
                                                               architecture = GPU(),
                                                               x = (0,Lx),
                                                               y = (0,Ly),
                                                               halo = (3, 3, 3),
                                                               z_faces = hyperbolically_spaced_nodes)

#Coefficient of Thermal expansion
const alpha= 2e-4

#Coefficient of Salinity
const saline=0
#diffusive viscocity
const v=1e-5 
#diffusivity
const k=2e-6  

const l=Lx/2  #center of gaussian field
const m=Ly/2  #center of gausian field

const Bo=3.6e-4 #maximum surface flux

const f=-0.5   #coriolis parameter

Name_0f_simulation = "VerticalStratificationWeno"
###############################################################


const Raf= (Bo*Lz^4)/(v*k^2)

const Ro= ((Bo/(-f)^3)^0.5)/Lz

const Ro_l= (Bo/(-f)^3*R0^2)^(1/4)

const N= (g*alpha*dTz)^0.5

const N_cap= N/(-f)



# Surface Forcing at the top
Q(x,y,t) = Bo*exp((-0.5)*(((x-l)/0.4)^2+((y-m)/0.4)^2));

const dTdz = 0 # K m⁻¹

T_bcs = TracerBoundaryConditions(computational_grid,
                                 top = FluxBoundaryCondition(Q),
                                 bottom = GradientBoundaryCondition(dTdz))


const Qᵘ=0    #Zero flux boundary condition at the top surface

u_bcs = UVelocityBoundaryConditions(computational_grid, top = FluxBoundaryCondition(Qᵘ), bottom=ValueBoundaryCondition(0.0))
v_bcs = VVelocityBoundaryConditions(computational_grid, top = FluxBoundaryCondition(Qᵘ), bottom=ValueBoundaryCondition(0.0))



buoyancy = SeawaterBuoyancy(gravitational_acceleration = g,equation_of_state=LinearEquationOfState(α=alpha, β=saline))


#Incompressible model initiation 

using Oceananigans.Advection
using Oceananigans.TurbulenceClosures

model = IncompressibleModel(architecture = GPU(),
                            advection = WENO5(),
                            timestepper = :RungeKutta3,
                            grid = computational_grid,
                            coriolis = FPlane(f=f),
                            buoyancy = buoyancy,
                            closure = SmagorinskyLilly(),
                            boundary_conditions = (u=u_bcs, v=v_bcs, T=T_bcs))


# Random noise damped at top and bottom
Ξ(z) = randn() * z / model.grid.Lz * (1 + z / model.grid.Lz) # noise

# Temperature initial condition: a stable density gradient with random noise superposed.


Tᵢ(x, y, z) = T0 + dTz * z + dTz * model.grid.Lz * Factor_T * Ξ(z)

# Velocity initial condition: random noise scaled by the friction velocity.
uᵢ(x, y, z) = sqrt(abs(Qᵘ)) * Factor_V * Ξ(z)

# `set!` the `model` fields using functions or constants:
set!(model, u=uᵢ, v=uᵢ, w=uᵢ, T=Tᵢ)


using Oceananigans.Diagnostics: accurate_cell_advection_timescale
wizard = TimeStepWizard(cfl=0.05,Δt=0.01, max_change=1.1, max_Δt=1minute,cell_advection_timescale = accurate_cell_advection_timescale)

start_time = time_ns() # so we can print the total elapsed wall time

# Print a progress message
progress_message(sim) =
    @printf("i: %04d, t: %s, Δt: %s, wmax = %.1e ms⁻¹, wall time: %s\n",
            sim.model.clock.iteration, prettytime(model.clock.time),
            prettytime(wizard), maximum(abs, sim.model.velocities.w),
            prettytime((time_ns() - start_time) * 1e-9))

simulation = Simulation(model,
                    Δt = wizard,
             stop_time = 10minutes,
    iteration_interval = 1,
              progress = progress_message
)


fields = Dict("u" => model.velocities.u,"v" => model.velocities.v,"w" => model.velocities.w, "T" => model.tracers.T)

simulation.output_writers[:fields] =
    NetCDFOutputWriter(model, fields, filepath="convection25.nc",
                       schedule=TimeInterval(6) )



run!(simulation)



# simulation.output_writers[:fields] =
#     JLD2OutputWriter(model, merge(model.velocities, model.tracers),
#                            prefix = Name_0f_simulation,
#                          schedule = TimeInterval(0.1minute),
#                             force = true)

# run!(simulation)


using JLD2

using Plots
using Printf



file = jldopen("VerticalStretchingWeno.jld2")


# Coordinate arrays
xC, yC, zC = file["grid/xᶜᵃᵃ"][4:259],file["grid/yᵃᶜᵃ"][4:259],file["grid/zᵃᵃᶜ"][4:68]

# Extract a vector of iterations
iterations = parse.(Int, keys(file["timeseries/t"]))

@info "Making a neat movie of verticle velocity and Temperature..."

anim = @animate for (i, iteration) in enumerate(iterations)

    @info "Plotting frame $i from iteration $iteration..."

    t = file["timeseries/t/$iteration"]
    u_snapshot = file["timeseries/u/$iteration"][:, :, Nz]
    v_snapshot = file["timeseries/v/$iteration"][:, :, Nz]
    w_snapshot = file["timeseries/w/$iteration"][:, 128, :]
    speed_snapshot = sqrt.(u_snapshot.*u_snapshot + v_snapshot.*v_snapshot)
  #  T_snapshot = file["timeseries/T/$iteration"][:, 128, :]
   

    ulims = 0.01
    
    ulevels = range(-ulims, stop=ulims, length=50)
    
    slims = 0.025
    
    slevels = range(0, stop=slims, length=50)
    

    kwargs1 = (xlabel="x", ylabel="z", aspectratio=50, linewidth=0, colorbar=true,
              xlims=(0, Lx), ylims=(-Lz,0))
    
    kwargs2 = (xlabel="x", ylabel="y", aspectratio=1, linewidth=0, colorbar=true,
              xlims=(0, Lx), ylims=(0, Ly))

    u_plot = contourf(xC, zC, clamp.(w_snapshot', -ulims, ulims);
                        color = :balance,
                        clims=(-ulims, ulims), levels=ulevels,
                        kwargs1...)

    s_plot = contourf(xC, yC, clamp.(speed_snapshot', -slims,slims );
                       color = :thermal,
                       clims=(0, slims), levels=slevels,
                       kwargs2...)
#title
    u_title = @sprintf("vertical velocity (m s⁻¹), t = %s", prettytime(t))
    s_title = "Horizontal speed"
    plot( u_plot, s_plot, title= [u_title s_title], layout=(1, 2), size=(1200, 600))
end



mp4(anim, "./animationWeno.mp4", fps=1) 



# Plotting MLD and comparision code

In [ ]:
file1 = jldopen("VerticalStretchingWeno.jld2")
file2 = jldopen("VerticalStretchingregular.jld2")

# Coordinate arrays
xC1, yC1, zC1 = file1["grid/xᶜᵃᵃ"][4:259],file1["grid/yᵃᶜᵃ"][4:259],file1["grid/zᵃᵃᶜ"][4:68]

# Extract a vector of iterations
iterations1 = parse.(Int, keys(file1["timeseries/t"]))
mixed_layer_depth1=zeros(length(iterations1))
T0 = 30
c=1

for i in iterations1
    
    for zc in 1:1:Nz
        h=0
        for xc in 64:1:192
            for yc in 64:1:192
                vertical_velocity = file1["timeseries/w/$i"][xc, yc, zc]
                Temperature = file1["timeseries/T/$i"][xc,yc,zc]
                
                h=h + ((vertical_velocity)*g*alpha*(Temperature-T0)*(Lx/Nx)*(Ly/Ny))/(Lx*Ly)
            end
        end
        print(h,c)
        
        if h >= 0.001*3.6e-4
            mixed_layer_depth1[c]=-Lz+(zc-1)*(Lz/Nz)
            print("worked")
            break
        else
            mixed_layer_depth1[c] =  mixed_layer_depth1[c]
            print("continue")
        end
    end
    c= c+1
end

iterations2 = parse.(Int, keys(file2["timeseries/t"]))
mixed_layer_depth2=zeros(length(iterations2))
T0 = 30
c=1

for i in iterations2
    
    for zc in 1:1:Nz
        h=0
        for xc in 1:1:Nx
            for yc in 1:1:Ny
                vertical_velocity = file2["timeseries/w/$i"][xc, yc, zc]
                Temperature = file2["timeseries/T/$i"][xc,yc,zc]
                
                h=h + ((vertical_velocity)*g*alpha*(Temperature-T0)*(Lx/Nx)*(Ly/Ny))/(Lx*Ly)
            end
        end
        print(h,c)
        
        if h >= 0.001*3.6e-4
            mixed_layer_depth2[c]=-Lz+(zc-1)*(Lz/Nz)
            print("worked")
            break
        else
            mixed_layer_depth2[c] =  mixed_layer_depth2[c]
            print("continue")
        end
    end
    c= c+1
end

x = 1:101
plot(x,[mixed_layer_depth1,mixed_layer_depth2],label=["With Stretching","Without Stretching"], yticks = [-1:0.1:0],xticks = [1:1:101])
savefig("plot3.png")

#yticks!([-1:0.1:0;], ["min", "zero", "max"])
#xaxis = ("Iterations", (1,101), 0:1:101, :log, font(20, "Courier"))
#yaxis = ("h", (-1,0), -1:0.1:0, :log, :flip, font(20, "Courier"))